# Analysis of Effects from Artilces of Posts

In [ ]:
import numpy as np
import pandas as pd
from scipy.stats import kruskal
from joblib import Parallel, delayed
import matplotlib.pyplot as plt

In [2]:
#set format to float with 4 decimals
pd.set_option('display.float_format', '{:.4f}'.format)

Load, specify and visualize data for analysis

In [ ]:
#read data
sentiments = pd.read_csv("sentiment_results.csv")

#emotions of articles(_A) and posts(_P) included in the analysis 
emotions_a = ['Anger_A', 'Fear_A', 'Disgust_A', 'Joy_A', 'None_A']
emotions_p = ['Anger_P', 'Fear_P', 'Disgust_P', 'Joy_P', 'None_P']

#filter relevant colums
sentiments = sentiments[emotions_a + ['NewsroomTopic'] + emotions_p]

#visualize distribution of predictors
sentiments[emotions_a].hist(bins=30, figsize=(12, 8))
plt.tight_layout()
plt.show()

Set Global Parameters

In [4]:
#set for optimal cpu utilisation
parallel_jobs = 10

#settings to account for testing errors
alpha = 0.05
bootstrap_samples = 10000

#ranking parameters for emotions in articles
between_ranks = 2
rank_threshold = 1/6

#set seed for reproducibility
seed = 666

Precompute Variables

In [5]:
#prepare topics
sentiments['NewsroomTopic'] = sentiments['NewsroomTopic'].astype('category')
topics = sentiments['NewsroomTopic'].to_numpy()
topic_levels = sentiments['NewsroomTopic'].cat.categories.to_numpy()


#convert DV arrays for faster access
dv_arrays = {dv: sentiments[dv].to_numpy() for dv in emotions_p}

#generate seed based bootstrap indices for tests
rng = np.random.default_rng(seed)
n = len(sentiments)

bootstrap_indices = []
for i in range(bootstrap_samples):
    idx = rng.choice(n, size=n, replace=True)
    bootstrap_indices.append(idx)

#build dicts to store bootstrap stats
boot_pvalues_dict = {}
boot_epsilon_dict = {}
boot_k_dict = {}
boot_desc_means_dict = {}
boot_desc_stds_dict = {}

Test Functions

In [6]:
def kruskal_test(groups):

    clean_groups = [g for g in groups if len(g) > 0]

    if len(clean_groups) <= 1:
        return np.nan, np.nan, np.nan, np.nan
    
    H, p = kruskal(*clean_groups)
    k = len(clean_groups)
    n_total = sum(len(g) for g in clean_groups)
    epsilon = (H - k + 1) / (n_total - k) if n_total > k else np.nan

    return H, p, epsilon, k

In [7]:
def bootstrap_stats(H_list, epsilon_list, p_list, k_list):
    
    H_arr = np.array(H_list)
    eps_arr = np.array(epsilon_list)
    p_arr = np.array(p_list)
    k_arr = np.array(k_list)
    
    return {
        "H_mean": np.nanmean(H_arr),
        "epsilon2_mean": np.nanmean(eps_arr),
        "epsilon2_ci_lower": np.nanpercentile(eps_arr, 100 * alpha / 2),
        "epsilon2_ci_upper": np.nanpercentile(eps_arr, 100 * (1 - alpha / 2)),
        "mean_p_value": np.nanmean(p_arr),
        "signif_prob": np.mean(p_arr <= alpha) if len(p_arr) > 0 else np.nan,
        "mean_k": np.nanmean(k_arr)
    }

Analysis of Emotions

In [8]:
def emotion_bootstrap(idx_list, dv, iv_emo):

    H_vals, epsilon_vals, p_vals, k_vals = [], [], [], []
    for idx in idx_list:
        dv_data = dv_arrays[dv][idx]
        iv_data = sentiments[iv_emo].to_numpy()[idx]
        topic_data = topics[idx]
        
        num_ranks = between_ranks + 2 if rank_threshold > 0 else between_ranks
        
        if rank_threshold > 0:
            edges = np.linspace(rank_threshold, 1 - rank_threshold, between_ranks + 1)
            edges = np.concatenate([[0], edges, [1]])
        else:
            edges = np.linspace(0, 1, between_ranks + 1)
        
        edges = np.unique(edges)
        rank_data = pd.cut(
            iv_data,
            bins=edges,
            labels=False,
            include_lowest=True,
            duplicates='drop'
        ) + 1
        
        for lvl in topic_levels:
            mask = topic_data == lvl
            groups = [dv_data[(mask) & (rank_data == r)] for r in range(1, len(edges))]
            
            H, p, epsilon, k = kruskal_test(groups)
            
            if not np.isnan(H):
                H_vals.append(H)
                epsilon_vals.append(epsilon)
                p_vals.append(p)
                k_vals.append(k)
    
    return {"H": H_vals, "epsilon": epsilon_vals, "p": p_vals, "k": k_vals}

In [9]:
def emotion_parallel(dv, iv_emo):
    
    split_indices = np.array_split(bootstrap_indices, parallel_jobs)
    results = Parallel(n_jobs=parallel_jobs)(
        delayed(emotion_bootstrap)(sub_idx, dv, iv_emo) for sub_idx in split_indices
    )

    H_all, eps_all, p_all, k_all = [], [], [], []
    for r in results:
        H_all.extend(r["H"])
        eps_all.extend(r["epsilon"])
        p_all.extend(r["p"])
        k_all.extend(r["k"])

    boot_epsilon_dict[(dv, iv_emo)] = eps_all
    boot_pvalues_dict[(dv, iv_emo)] = p_all
    boot_k_dict[(dv, iv_emo)] = k_all

    return bootstrap_stats(H_all, eps_all, p_all, k_all)

Analysis of Topics

In [10]:
def topic_bootstrap(idx_list, dv):

    H_vals, epsilon_vals, p_vals, k_vals = [], [], [], []
    for idx in idx_list:
        dv_data = dv_arrays[dv][idx]
        topic_data = topics[idx]
        groups = [dv_data[topic_data == lvl] for lvl in topic_levels]
        H, p, epsilon, k = kruskal_test(groups)
        
        if not np.isnan(H):
            H_vals.append(H)
            epsilon_vals.append(epsilon)
            p_vals.append(p)
            k_vals.append(k)

    return {"H": H_vals, "epsilon": epsilon_vals, "p": p_vals, "k": k_vals}

In [11]:
def topic_parallel(dv):
    
    split_indices = np.array_split(bootstrap_indices, parallel_jobs)
    results = Parallel(n_jobs=parallel_jobs)(
        delayed(topic_bootstrap)(sub_idx, dv) for sub_idx in split_indices
    )

    H_all, eps_all, p_all, k_all = [], [], [], []
    for r in results:
        H_all.extend(r["H"])
        eps_all.extend(r["epsilon"])
        p_all.extend(r["p"])
        k_all.extend(r["k"])

    boot_epsilon_dict[(dv, "NewsroomTopic")] = eps_all
    boot_pvalues_dict[(dv, "NewsroomTopic")] = p_all
    boot_k_dict[(dv, "NewsroomTopic")] = k_all

    return bootstrap_stats(H_all, eps_all, p_all, k_all)

Descriptive statistics

In [12]:
def descriptive_stats():

    numeric_vars = emotions_a + emotions_p
    topics_arr = sentiments["NewsroomTopic"].to_numpy()

    rows = []
    for var in numeric_vars:
        x = sentiments[var].to_numpy()

        for lvl in topic_levels:
            topic_mask = topics_arr == lvl
            data = x[topic_mask]

            if data.size == 0:
                continue

            mean = np.mean(data)
            std = np.std(data, ddof=1)
            boot_means = np.empty(bootstrap_samples)

            for b in range(bootstrap_samples):
                sample_idx = bootstrap_indices[b]
                sample_data = x[sample_idx][topic_mask[sample_idx]]
                boot_means[b] = (
                    np.mean(sample_data) if sample_data.size > 0 else np.nan
                )

            mean_ci_lower = np.nanpercentile(boot_means, 100 * alpha / 2)
            mean_ci_upper = np.nanpercentile(boot_means, 100 * (1 - alpha / 2))
            boot_desc_means_dict[(var, lvl)] = boot_means
            rows.append({
                "variable": var,
                "topic": lvl,
                "mean": mean,
                "mean_ci_lower": mean_ci_lower,
                "mean_ci_upper": mean_ci_upper,
                "std": std
            })

    return pd.DataFrame(rows)

Summarize Testings with Structured Output

In [13]:
def kruskal_wallis_bootstrap():

    test_rows = []
    for dv in emotions_p:

        for iv_emo in emotions_a:
            stats = emotion_parallel(dv, iv_emo)
            signif_label = "yes" if stats["epsilon2_ci_lower"] > 0 else "no"
            test_rows.append({
                "criterion": dv,
                "predictor": iv_emo,
                "type": "emotion",
                "signif_label": signif_label,
                **stats
            })

        stats = topic_parallel(dv)
        signif_label = "yes" if stats["epsilon2_ci_lower"] > 0 else "no"
        test_rows.append({
            "criterion": dv,
            "predictor": "NewsroomTopic",
            "type": "NewsroomTopic",
            "signif_label": signif_label,
            **stats
        })

    test_df = pd.DataFrame(test_rows)
    desc_df = descriptive_stats()

    return test_df, desc_df

# Run Analysis

In [14]:
test_stats, desc_stats = kruskal_wallis_bootstrap()

Visualize Bootstrap Distributions

In [29]:
def plot_bootstrap_distribution(stat_type, criterion=None, predictor=None, variable=None, topic=None):
    
    if stat_type in ["H", "epsilon", "p"]:
        if criterion is None or predictor is None:
            raise ValueError("criterion and predictor must be provided for Kruskal-Wallis stats")
        elif stat_type == "epsilon":
            data = boot_epsilon_dict.get((criterion, predictor), [])
            title = f"Epsilon² distribution for {criterion} ~ {predictor}"
        else:
            data = boot_pvalues_dict.get((criterion, predictor), [])
            title = f"p-value distribution for {criterion} ~ {predictor}"
    elif stat_type in ["mean", "std"]:
        if variable is None or topic is None:
            raise ValueError("variable and topic must be provided for descriptives")
        if stat_type == "mean":
            data = boot_desc_means_dict.get((variable, topic), [])
            title = f"Bootstrap means for {variable} in topic {topic}"
        else:
            data = boot_desc_stds_dict.get((variable, topic), [])
            title = f"Bootstrap stds for {variable} in topic {topic}"
    else:
        raise ValueError("stat_type must be one of H, epsilon, p, mean, std")
    
    if len(data) == 0:
        raise ValueError("No bootstrap data found for the given inputs.")
    
    fig = go.Figure()
    fig.add_trace(go.Histogram(
        x=data,
        nbinsx=50,
        histnorm='probability density',
        marker_color='lightblue',
        opacity=0.75
    ))

    fig.update_layout(
        title=title,
        xaxis_title=stat_type,
        yaxis_title="Density",
        template="plotly_white",
        bargap=0.2
    )
    fig.show()

In [ ]:
#stats: epsilon, p, mean
plot_bootstrap_distribution(stat_type="epsilon", criterion="Anger_P", predictor="Anger_A")

Save test_stats as CSV

In [ ]:
#test_stats.to_csv('test_stats.csv', index=False)

In [ ]:
#desc_stats.to_csv('descriptive_stats.csv', index=False)